In [1]:
companies = ['GE', 'F', 'AMD', 'FIT', 'GPRO', 'SNAP', 'CRON', 'CHK', 'SIRI', 'XXII', 'ZNGA', 'S', 'GRPN', 'RAD',
             'PLUG', 'HMNY', 'NVAX', 'INSY', 'UAA', 'APRN', 'GLUU', 'TRXC', 'AUY', 'BOTZ', 'GERN', 'BILI', 'JCP',
            'CARA', 'VSLR', 'P', 'TEVA', 'RUN', 'UA', 'NOK', 'SPWR', 'CY']

In [8]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

# Import the backtrader platform
import backtrader as bt

import pandas as pd
from pandas_datareader import data, wb
import datetime as dt
from fbprophet import Prophet
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

In [131]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # keep array of dates and closes
        self.date_array = []
        self.close_array = []

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        #self.sma = bt.indicators.SimpleMovingAverage(
        #    self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.datas[0].close[0])
        
        # append date and close to arrays
        self.date_array.append(self.datas[0].datetime.date(0))
        self.close_array.append(self.datas[0].close[0])
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # make sure we have a decent amount of data
        if len(self.date_array) < 90:
            return
        
        # only invest once a week
        if len(self.date_array) % 5 != 0:
            return
        
        # get predictions
        max_move, expected_move, min_move = self.get_prophet_moves(7, False)
        
        if max_move > 0 and abs(max_move) > abs(min_move): # buy
            # BUY, BUY, BUY!!! (with default parameters)
            self.log('BUY CREATE, %.2f' % self.datas[0].close[0])

            # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()
        elif min_move < 0 and abs(min_move) > abs(max_move): # sell
            if self.position:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.datas[0].close[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
    def get_prophet_moves(self, daysOut=7, showCharts=False):
        # create stock dataframe for prophet
        stock_df = pd.DataFrame({
            'ds': self.date_array,
            'y': self.close_array
        })

        # fit data using prophet model
        m = Prophet()
        m.fit(stock_df)

        # create future dates
        future_prices = m.make_future_dataframe(periods=365)

        # predict prices
        forecast = m.predict(future_prices)

        # view results
        if showCharts:
            fig = m.plot(forecast)
            ax1 = fig.add_subplot(111)
            ax1.set_title("Stock Price Forecast", fontsize=16)
            ax1.set_xlabel("Date", fontsize=12)
            ax1.set_ylabel("Close Price", fontsize=12)

            fig2 = m.plot_components(forecast)
            plt.show()

        # calculate predicted returns
        end_of_period = self.datas[0].datetime.date(0) + dt.timedelta(days=daysOut)

        future_close_max = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat_upper
        future_close_expected = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat
        future_close_min = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat_lower

        # calculate percent changes based on predictions
        max_move = (future_close_max - self.datas[0].close[0])/self.datas[0].close[0]
        expected_move = (future_close_expected - self.datas[0].close[0])/self.datas[0].close[0]
        min_move = (future_close_min - self.datas[0].close[0])/self.datas[0].close[0]

        return (max_move, expected_move, min_move)

In [132]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Create a Data Feed
data = bt.feeds.YahooFinanceData(
    dataname='GE',
    fromdate=dt.datetime(2000, 1, 1),
    todate=dt.datetime(2017, 12, 31),
    reverse=False)

cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(10000.0)# Add the Data Feed to Cerebro

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1)

# Set the commission
cerebro.broker.setcommission(commission=0.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00


INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2000-01-03, Close, 27.64
2000-01-04, Close, 26.53
2000-01-05, Close, 26.49
2000-01-06, Close, 26.84
2000-01-07, Close, 27.88
2000-01-10, Close, 27.87
2000-01-11, Close, 27.92
2000-01-12, Close, 28.01
2000-01-13, Close, 28.33
2000-01-14, Close, 27.82
2000-01-18, Close, 27.27
2000-01-19, Close, 27.40
2000-01-20, Close, 26.89
2000-01-21, Close, 26.56
2000-01-24, Close, 25.45
2000-01-25, Close, 25.52
2000-01-26, Close, 26.06
2000-01-27, Close, 26.12
2000-01-28, Close, 24.69
2000-01-31, Close, 24.69
2000-02-01, Close, 25.06
2000-02-02, Close, 24.70
2000-02-03, Close, 25.66
2000-02-04, Close, 26.08
2000-02-07, Close, 25.15
2000-02-08, Close, 25.26
2000-02-09, Close, 24.70
2000-02-10, Close, 24.96
2000-02-11, Close, 24.65
2000-02-14, Close, 24.76
2000-02-15, Close, 25.34
2000-02-16, Close, 25.00
2000-02-17, Close, 24.14
2000-02-18, Close, 23.06
2000-02-22, Close, 23.89
2000-02-23, Close, 24.05
2000-02-24, Close, 24.14
2000-02-25, Close, 23.24
2000-02-28, Close, 23.85
2000-02-29, Close, 24.39


/home/sean/.virtualenvs/ml/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2000-05-10, BUY CREATE, 28.23
2000-05-11, BUY EXECUTED, Price: 51.50, Cost: 51.50, Comm 0.00
2000-05-11, Close, 28.41
2000-05-12, Close, 29.14
2000-05-15, Close, 30.12
2000-05-16, Close, 30.26
2000-05-17, Close, 29.91


...

2017-12-04, BUY CREATE, 17.41
2017-12-05, BUY EXECUTED, Price: 17.78, Cost: 17.78, Comm 0.00
2017-12-05, Close, 17.22
2017-12-06, Close, 17.24
2017-12-07, Close, 17.29
2017-12-08, Close, 17.29
2017-12-11, Close, 17.23


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2017-12-11, BUY CREATE, 17.23
2017-12-12, BUY EXECUTED, Price: 17.95, Cost: 17.95, Comm 0.00
2017-12-12, Close, 17.49
2017-12-13, Close, 17.34
2017-12-14, Close, 17.22
2017-12-15, Close, 17.40
2017-12-18, Close, 17.34


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2017-12-18, BUY CREATE, 17.34
2017-12-19, BUY EXECUTED, Price: 17.77, Cost: 17.77, Comm 0.00
2017-12-19, Close, 17.17
2017-12-20, Close, 17.04
2017-12-21, Close, 17.06
2017-12-22, Close, 17.09
2017-12-26, Close, 17.13
2017-12-26, BUY CREATE, 17.13
2017-12-27, BUY EXECUTED, Price: 17.46, Cost: 17.46, Comm 0.00
2017-12-27, Close, 17.09
2017-12-28, Close, 17.07
2017-12-29, Close, 17.15
Final Portfolio Value: 8578.94
